In [6]:
import torch
import pandas as pd
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
tqdm.pandas()

In [7]:
read = pd.read_pickle('Pickle/read.pkl')
books = pd.read_pickle('Pickle/books.pkl')
reviews = pd.read_pickle('Pickle/reviews.pkl')
interactions = pd.read_pickle('Pickle/interactions.pkl')    

In [8]:
interactions = interactions[interactions['is_read']== True]
reviews = reviews.dropna(subset=['review_text'])
# Get unique user IDs from both DataFrames
interaction_user_ids = set(interactions['user_id'].unique())
review_user_ids = set(reviews['user_id'].unique())
# Find the common user IDs
common_user_ids = interaction_user_ids.intersection(review_user_ids)

In [9]:
# Filter interactions DataFrame
interactions = interactions[interactions['user_id'].isin(common_user_ids)]
# Filter reviews DataFrame
reviews = reviews[reviews['user_id'].isin(common_user_ids)]

In [15]:
embeddings_file = pd.read_pickle('Pickle/review_embeddings.pkl')

EOFError: Ran out of input

In [12]:
import os

model = SentenceTransformer('all-MiniLM-L6-v2')

def save_embeddings_incrementally(reviews_df, model, interval=100):
    embeddings_file = 'Pickle/review_embeddings.pkl'
    
    if os.path.exists(embeddings_file):
        embeddings_df = pd.read_pickle(embeddings_file)
    else:
        embeddings_df = pd.DataFrame(columns=['index', 'embeddings'])
    
    reviews_df = reviews_df.dropna(subset=['review_text']).reset_index(drop=True)

    for i in tqdm(range(len(reviews_df))):
        if i in embeddings_df['index'].values:
            continue  # Skip if already processed
        
        embedding = model.encode(reviews_df.loc[i, 'review_text'])
        new_row = pd.DataFrame({'index': [i], 'embeddings': [embedding]})
        embeddings_df = pd.concat([embeddings_df, new_row], ignore_index=True)
        
        if i % interval == 0:
            embeddings_df.to_pickle(embeddings_file)
    
    # Save the final version
    embeddings_df.to_pickle(embeddings_file)

# Save embeddings incrementally
save_embeddings_incrementally(reviews, model, interval=100)


c:\Users\maddy\anaconda3\envs\goodreads\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


EOFError: Ran out of input

In [ ]:
# Load the incremental embeddings
embeddings_df = pd.read_pickle('Pickle/review_embeddings.pkl')

# Ensure the reviews DataFrame has a proper index
reviews.reset_index(drop=True, inplace=True)

# Initialize the embeddings column in reviews DataFrame
reviews['embeddings'] = None

# Merge embeddings back into the reviews DataFrame
for i in tqdm(range(len(reviews))):
    if i in embeddings_df['index'].values:
        embedding = embeddings_df.loc[embeddings_df['index'] == i, 'embeddings'].values[0]
        reviews.at[i, 'embeddings'] = embedding

# Ensure all embeddings are numpy arrays
def convert_to_array(x):
    if isinstance(x, list):
        return np.array(x)
    return x

reviews['embeddings'] = reviews['embeddings'].apply(convert_to_array)

In [ ]:
def calculate_weighted_embeddings(reviews_df, base_weight=0.1):
    # Normalize n_votes and n_comments
    reviews_df['n_votes_normalized'] = reviews_df['n_votes'] / reviews_df['n_votes'].max()
    reviews_df['n_comments_normalized'] = reviews_df['n_comments'] / reviews_df['n_comments'].max()

    # Calculate weights with a base weight
    reviews_df['weight'] = base_weight + reviews_df['n_votes_normalized'] + reviews_df['n_comments_normalized']

    # Apply weights to embeddings
    reviews_df['weighted_embeddings'] = reviews_df.apply(lambda row: row['embeddings'] * row['weight'], axis=1)
    return reviews_df

# Apply the function
reviews = calculate_weighted_embeddings(reviews)


In [ ]:
user_embeddings = reviews.groupby('user_id')['weighted_embeddings'].apply(lambda x: np.mean(np.vstack(x.dropna()), axis=0)).reset_index()

In [ ]:

# Encode user and book IDs
user_encoder = LabelEncoder()
book_encoder = LabelEncoder()
interactions['user_id_enc'] = user_encoder.fit_transform(interactions['user_id'])
interactions['book_id_enc'] = book_encoder.fit_transform(interactions['book_id'])

# Create edge index (user-book pairs)
edge_index = torch.tensor([interactions['user_id_enc'].values, interactions['book_id_enc'].values], dtype=torch.long)

# Create edge attributes (ratings)
edge_attr = torch.tensor(interactions['rating'].values, dtype=torch.float)

# Create node features (using your user and book embeddings)
# Assuming user_features and book_features are numpy arrays
user_features = torch.tensor(user_embeddings['weighted_embeddings'].values, dtype=torch.float)
book_features = torch.tensor(books['some_feature_column'].values, dtype=torch.float)

# Combine user and book features into one feature matrix
node_features = torch.cat([user_features, book_features], dim=0)

# Define the graph data
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x


In [ ]:
from torch_geometric.loader import DataLoader

# Create a DataLoader
data_list = [data]  # If you have multiple graphs, create a list
loader = DataLoader(data_list, batch_size=1, shuffle=True)

# Initialize the model, loss function, and optimizer
model = GNN(input_dim=node_features.size(1), hidden_dim=64, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Training loop
model.train()
for epoch in range(100):  # Adjust the number of epochs as needed
    for batch in loader:
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.edge_attr)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')
